# **Real-ESRGAN Standalone Upscaler (Fast Mode)**

A fast upscaling solution using Real-ESRGAN/ESRGAN models directly without diffusion.

## Speed Comparison
- **This notebook**: ~10-60 seconds per image (4x upscale)
- **FLUX notebook**: ~10-20 minutes per image (4x upscale)

## When to Use
- Fast batch processing
- When original image quality is already good
- Preview/proof before final FLUX upscale
- When you don't need AI-generated detail enhancement

## Available Models
- `4x-UltraSharp.pth` - Sharp details, good for digital art (default)
- `4x_foolhardy_Remacri.pth` - Natural textures, less over-sharpening
- `4x-AnimeSharp.pth` - Optimized for anime/illustration

In [1]:
# @title Setup Environment (Run Once)
import subprocess
import sys
import os
from pathlib import Path

# Get repo root
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

# Directory paths
HF_DIR = REPO_ROOT / "hf"
MODELS_DIR = HF_DIR / "models"
UPSCALE_MODELS_DIR = MODELS_DIR / "upscale_models"
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"

# Create directories
for d in [HF_DIR, MODELS_DIR, UPSCALE_MODELS_DIR, OUTPUT_DIR, INPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Models directory: {UPSCALE_MODELS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

def install_packages():
    """Install required packages for Real-ESRGAN standalone."""
    # PyTorch with CUDA 12.8 for RTX 5060 Ti
    print("Installing PyTorch nightly with CUDA 12.8...")
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-q', '--pre',
         'torch', 'torchvision',
         '--index-url', 'https://download.pytorch.org/whl/nightly/cu128'],
        capture_output=True
    )
    if result.returncode == 0:
        print("✓ PyTorch installed")
    else:
        print(f"✗ PyTorch install failed: {result.stderr.decode()}")
    
    # Spandrel for loading upscale models
    packages = ['spandrel', 'opencv-python', 'huggingface_hub', 'safetensors', 'pillow']
    for package in packages:
        try:
            subprocess.run(
                [sys.executable, '-m', 'pip', 'install', '-q', package],
                check=True, capture_output=True
            )
            print(f"✓ {package} installed")
        except subprocess.CalledProcessError as e:
            print(f"✗ Error installing {package}")

print("Installing packages...")
install_packages()
print("\n✅ Environment Setup Complete!")

Repo root: C:\Users\Armaan\Desktop\Artinafti
Models directory: C:\Users\Armaan\Desktop\Artinafti\hf\models\upscale_models
Output directory: C:\Users\Armaan\Desktop\Artinafti\4xoutput
Installing packages...
Installing PyTorch nightly with CUDA 12.8...
✓ PyTorch installed
✓ spandrel installed
✓ opencv-python installed
✓ huggingface_hub installed
✓ safetensors installed
✓ pillow installed

✅ Environment Setup Complete!


In [2]:
# @title Download Models (Run Once)
from huggingface_hub import hf_hub_download
from pathlib import Path
import os

# Ensure paths are set
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR
UPSCALE_MODELS_DIR = REPO_ROOT / "hf" / "models" / "upscale_models"
UPSCALE_MODELS_DIR.mkdir(parents=True, exist_ok=True)

def download_model(repo_id: str, filename: str, dest_dir: Path) -> str:
    """Download model from HuggingFace Hub."""
    dest_path = dest_dir / filename
    if dest_path.exists():
        print(f"✓ {filename} already exists")
        return filename
    
    try:
        print(f"Downloading {filename}...", end=' ', flush=True)
        hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            local_dir=str(dest_dir),
            local_dir_use_symlinks=False
        )
        print("Done!")
        return filename
    except Exception as e:
        print(f"\nError downloading {filename}: {e}")
        return None

print("Downloading upscale models...")
download_model("Isi99999/Upscalers", "4x-UltraSharp.pth", UPSCALE_MODELS_DIR)
download_model("Isi99999/Upscalers", "4x_foolhardy_Remacri.pth", UPSCALE_MODELS_DIR)
download_model("Isi99999/Upscalers", "4x-AnimeSharp.pth", UPSCALE_MODELS_DIR)

print("\n✅ Models downloaded!")

✓ 4x-UltraSharp.pth already exists
✓ 4x_foolhardy_Remacri.pth already exists
✓ 4x-AnimeSharp.pth already exists

✅ Models downloaded!


In [3]:
# @title Load Libraries
import torch
import numpy as np
from PIL import Image
import cv2
import gc
import os
import time
from pathlib import Path
from IPython.display import display, Image as IPImage

# Import spandrel for model loading
from spandrel import ImageModelDescriptor, ModelLoader

# Set up paths
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

UPSCALE_MODELS_DIR = REPO_ROOT / "hf" / "models" / "upscale_models"
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Device setup
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def clear_memory():
    """Clear GPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

print(f"✅ Libraries loaded!")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

✅ Libraries loaded!
CUDA available: True
GPU: NVIDIA GeForce RTX 5060 Ti
VRAM: 15.9 GB


In [4]:
# @title Configuration - Batch Processing with Custom Output Sizes

# ============== BATCH INPUT WITH CUSTOM OUTPUT SIZES ==============
# Each image has its own target print size (width x height in inches)
# The upscaler will:
#   1. Upscale with Real-ESRGAN (4x)
#   2. Resize to exact target dimensions for printing

DPI = 150

IMAGE_CONFIGS = [
    {
        "image_id": "b46bc519-9b4b-487f-b8d4-b95195d7e02e",
        "input_path": "input/b46bc519-9b4b-487f-b8d4-b95195d7e02e.jpg",
        "width_inches": 10,
        "height_inches": 20,
    },
]

# Use 6x upscaling (two passes of 4x model, then downscale)
USE_6X_UPSCALE = True

# Calculate target pixel dimensions
for config in IMAGE_CONFIGS:
    config["target_width"] = config["width_inches"] * DPI
    config["target_height"] = config["height_inches"] * DPI

# ============== UPSCALE SETTINGS ==============
# Model options: "4x-UltraSharp.pth", "4x_foolhardy_Remacri.pth", "4x-AnimeSharp.pth"
upscale_model = "4x-UltraSharp.pth"

# Tile size for processing (larger = faster but more VRAM)
# For 16GB VRAM: 512-768 is safe
# For 8GB VRAM: 256-384 recommended
tile_size = 512
tile_overlap = 32  # Overlap between tiles to reduce seams

# Use FP16 for faster processing (recommended for RTX cards)
use_fp16 = True

print("Configuration set!")
print(f"Model: {upscale_model}")
print(f"Tile size: {tile_size}")
print(f"DPI: {DPI}")
print(f"\nImages to process:")
for config in IMAGE_CONFIGS:
    print(f"  {config['image_id'][:8]}...: {config['width_inches']}x{config['height_inches']}\" -> {config['target_width']}x{config['target_height']}px")

Configuration set!
Model: 4x-UltraSharp.pth
Tile size: 512
DPI: 150

Images to process:
  b46bc519...: 10x20" -> 1500x3000px


In [5]:
# @title Upscale Functions

def upscale_with_tiles(model, img_tensor: torch.Tensor, tile_size: int, tile_overlap: int) -> torch.Tensor:
    """
    Upscale image using tiled processing to handle large images.
    
    Args:
        model: Loaded spandrel model
        img_tensor: Input image tensor (B, C, H, W)
        tile_size: Size of each tile
        tile_overlap: Overlap between tiles
    
    Returns:
        Upscaled image tensor
    """
    scale = model.scale
    _, _, h, w = img_tensor.shape
    
    # If image is small enough, process directly
    if h <= tile_size and w <= tile_size:
        with torch.no_grad():
            return model(img_tensor)
    
    # Calculate output size
    out_h, out_w = h * scale, w * scale
    output = torch.zeros((1, 3, out_h, out_w), device=img_tensor.device, dtype=img_tensor.dtype)
    weight = torch.zeros((1, 1, out_h, out_w), device=img_tensor.device, dtype=img_tensor.dtype)
    
    # Calculate tile positions
    stride = tile_size - tile_overlap
    h_tiles = max(1, (h - tile_overlap) // stride + (1 if (h - tile_overlap) % stride else 0))
    w_tiles = max(1, (w - tile_overlap) // stride + (1 if (w - tile_overlap) % stride else 0))
    
    total_tiles = h_tiles * w_tiles
    print(f"Processing {total_tiles} tiles ({h_tiles}x{w_tiles})...")
    
    tile_count = 0
    for i in range(h_tiles):
        for j in range(w_tiles):
            # Calculate tile boundaries
            y1 = min(i * stride, h - tile_size) if h > tile_size else 0
            x1 = min(j * stride, w - tile_size) if w > tile_size else 0
            y2 = min(y1 + tile_size, h)
            x2 = min(x1 + tile_size, w)
            
            # Extract and process tile
            tile = img_tensor[:, :, y1:y2, x1:x2]
            
            with torch.no_grad():
                tile_out = model(tile)
            
            # Calculate output positions
            out_y1, out_y2 = y1 * scale, y2 * scale
            out_x1, out_x2 = x1 * scale, x2 * scale
            
            # Create weight mask for blending
            tile_h, tile_w = tile_out.shape[2:]
            mask = torch.ones((1, 1, tile_h, tile_w), device=tile_out.device, dtype=tile_out.dtype)
            
            # Feather edges for blending
            feather = tile_overlap * scale // 2
            if feather > 0:
                # Top edge
                if i > 0:
                    for k in range(feather):
                        mask[:, :, k, :] *= k / feather
                # Bottom edge
                if i < h_tiles - 1:
                    for k in range(feather):
                        mask[:, :, -(k+1), :] *= k / feather
                # Left edge
                if j > 0:
                    for k in range(feather):
                        mask[:, :, :, k] *= k / feather
                # Right edge
                if j < w_tiles - 1:
                    for k in range(feather):
                        mask[:, :, :, -(k+1)] *= k / feather
            
            # Add to output with blending
            output[:, :, out_y1:out_y2, out_x1:out_x2] += tile_out * mask
            weight[:, :, out_y1:out_y2, out_x1:out_x2] += mask
            
            tile_count += 1
            print(f"  Processed {tile_count}/{total_tiles} tiles", end='\r')
    
    print(f"  Processed {tile_count}/{total_tiles} tiles")
    
    # Normalize by weights
    output = output / weight.clamp(min=1e-8)
    
    return output


def upscale_and_resize(
    image_path: str,
    target_width: int,
    target_height: int,
    output_name: str,
    model_name: str,
    tile_size: int,
    tile_overlap: int,
    use_fp16: bool,
    use_two_pass: bool
) -> str:
    """
    Upscale image with Real-ESRGAN and resize to exact target dimensions.
    
    Args:
        image_path: Path to input image
        target_width: Target width in pixels
        target_height: Target height in pixels
        output_name: Output filename (without extension)
        model_name: Name of upscale model
        tile_size: Tile size for processing
        tile_overlap: Overlap between tiles
        use_fp16: Use FP16 precision
        use_two_pass: If True, run 4x model twice for 16x total (effective 6x+)
    
    Returns:
        Path to output image
    """
    start_time = time.time()
    
    # Resolve image path
    image_path = Path(image_path)
    if not image_path.is_absolute():
        image_path = REPO_ROOT / image_path
    
    if not image_path.exists():
        raise FileNotFoundError(f"Input file not found: {image_path}")
    
    print(f"Processing: {image_path.name}")
    print(f"Target output: {target_width}x{target_height}px")
    
    # Load image
    print("Loading image...")
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    print(f"Input size: {w}x{h}")
    
    # Calculate effective scale
    effective_scale = 16 if use_two_pass else 4
    upscaled_w, upscaled_h = w * effective_scale, h * effective_scale
    
    if use_two_pass:
        print(f"Strategy: Two-pass 4x upscale (16x total: {upscaled_w}x{upscaled_h}) then downscale to target")
    else:
        if upscaled_w >= target_width and upscaled_h >= target_height:
            print(f"Strategy: Upscale 4x ({upscaled_w}x{upscaled_h}) then downscale to target")
        else:
            print(f"⚠️ Warning: 4x upscale ({upscaled_w}x{upscaled_h}) may not reach target size")
    
    # Convert to tensor
    img_tensor = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    img_tensor = img_tensor.to(DEVICE)
    
    if use_fp16 and DEVICE.type == "cuda":
        img_tensor = img_tensor.half()
    
    # Load model
    print(f"Loading model: {model_name}")
    model_path = UPSCALE_MODELS_DIR / model_name
    if not model_path.exists():
        raise FileNotFoundError(f"Model not found: {model_path}")
    
    model = ModelLoader().load_from_file(str(model_path))
    assert isinstance(model, ImageModelDescriptor), "Not an image model!"
    
    model = model.to(DEVICE)
    if use_fp16 and DEVICE.type == "cuda":
        model = model.half()
    model.eval()
    
    scale = model.scale
    print(f"Model scale: {scale}x")
    
    # First pass upscale
    print("Upscaling (pass 1 of {})...".format(2 if use_two_pass else 1))
    upscale_start = time.time()
    
    output_tensor = upscale_with_tiles(model, img_tensor, tile_size, tile_overlap)
    
    pass1_time = time.time() - upscale_start
    print(f"Pass 1 took: {pass1_time:.1f}s")
    
    # Second pass if requested
    if use_two_pass:
        print("Upscaling (pass 2 of 2)...")
        pass2_start = time.time()
        
        # Use smaller tile size for second pass (image is now 4x larger)
        tile_size_pass2 = min(tile_size, 384)  # Smaller tiles for larger image
        output_tensor = upscale_with_tiles(model, output_tensor, tile_size_pass2, tile_overlap)
        
        pass2_time = time.time() - pass2_start
        print(f"Pass 2 took: {pass2_time:.1f}s")
    
    total_upscale_time = time.time() - upscale_start
    print(f"Total upscaling took: {total_upscale_time:.1f}s")
    
    # Convert back to image
    output = output_tensor.squeeze(0).permute(1, 2, 0).float().cpu().numpy()
    output = (output * 255).clip(0, 255).astype(np.uint8)
    
    upscaled_h, upscaled_w = output.shape[:2]
    print(f"Upscaled size: {upscaled_w}x{upscaled_h}")
    
    # Resize to exact target dimensions using high-quality Lanczos
    print(f"Resizing to target: {target_width}x{target_height}...")
    pil_img = Image.fromarray(output)
    pil_img = pil_img.resize((target_width, target_height), Image.LANCZOS)
    
    # Convert back to numpy for saving with cv2
    output = np.array(pil_img)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    
    # Save output
    output_path = OUTPUT_DIR / f"{output_name}.png"
    cv2.imwrite(str(output_path), output)
    
    # Cleanup
    del model, img_tensor, output_tensor
    clear_memory()
    
    total_time = time.time() - start_time
    
    print(f"\n✅ Done!")
    print(f"Output: {output_path}")
    print(f"Final size: {target_width}x{target_height}")
    print(f"Total time: {total_time:.1f}s")
    
    return str(output_path)

print("✅ Functions defined!")

✅ Functions defined!


In [6]:
# @title Run Batch Upscaling

print("="*60)
print("BATCH UPSCALING WITH CUSTOM OUTPUT SIZES")
print("="*60)

results = []

for i, config in enumerate(IMAGE_CONFIGS, 1):
    print(f"\n{'='*60}")
    print(f"[{i}/{len(IMAGE_CONFIGS)}] {config['image_id']}")
    print(f"Target: {config['width_inches']}x{config['height_inches']}\" at {DPI} DPI")
    print(f"{'='*60}")
    
    try:
        output_name = f"{config['image_id']}_real-esrgan"
        
        output_path = upscale_and_resize(
            image_path=config["input_path"],
            target_width=config["target_width"],
            target_height=config["target_height"],
            output_name=output_name,
            model_name=upscale_model,
            tile_size=tile_size,
            tile_overlap=tile_overlap,
            use_fp16=use_fp16,
            use_two_pass=USE_6X_UPSCALE
        )
        
        results.append({
            "image_id": config["image_id"],
            "status": "Success",
            "output": output_path,
            "size": f"{config['width_inches']}x{config['height_inches']}\""
        })
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        results.append({
            "image_id": config["image_id"],
            "status": "Failed",
            "error": str(e)
        })

# Summary
print(f"\n\n{'='*60}")
print("BATCH COMPLETE - SUMMARY")
print(f"{'='*60}")

success_count = sum(1 for r in results if r["status"] == "Success")
print(f"\nProcessed: {success_count}/{len(results)} images successfully\n")

for r in results:
    if r["status"] == "Success":
        print(f"✅ {r['image_id'][:8]}... -> {r['size']} -> {Path(r['output']).name}")
    else:
        print(f"❌ {r['image_id'][:8]}... -> {r.get('error', 'Unknown error')}")

BATCH UPSCALING WITH CUSTOM OUTPUT SIZES

[1/1] b46bc519-9b4b-487f-b8d4-b95195d7e02e
Target: 10x20" at 150 DPI
Processing: b46bc519-9b4b-487f-b8d4-b95195d7e02e.jpg
Target output: 1500x3000px
Loading image...
Input size: 1024x587
Strategy: Two-pass 4x upscale (16x total: 16384x9392) then downscale to target
Loading model: 4x-UltraSharp.pth
Model scale: 4x
Upscaling (pass 1 of 2)...
Processing 6 tiles (2x3)...
  Processed 6/6 tiles
Pass 1 took: 4.1s
Upscaling (pass 2 of 2)...
Processing 84 tiles (7x12)...
  Processed 84/84 tiles
Pass 2 took: 28.6s
Total upscaling took: 32.7s
Upscaled size: 16384x9392
Resizing to target: 1500x3000...

✅ Done!
Output: C:\Users\Armaan\Desktop\Artinafti\4xoutput\b46bc519-9b4b-487f-b8d4-b95195d7e02e_real-esrgan.png
Final size: 1500x3000
Total time: 37.6s


BATCH COMPLETE - SUMMARY

Processed: 1/1 images successfully

✅ b46bc519... -> 10x20" -> b46bc519-9b4b-487f-b8d4-b95195d7e02e_real-esrgan.png


## Single Image Processing (Optional)

Use the cell below if you want to process a single image with custom settings.

In [7]:
# @title Single Image Upscale (Optional)

# Uncomment and modify to process a single image:

# single_result = upscale_and_resize(
#     image_path="input/your_image.jpg",
#     target_width=1650,   # 11 inches * 150 DPI
#     target_height=2100,  # 14 inches * 150 DPI
#     output_name="your_image_real-esrgan",
#     model_name="4x-UltraSharp.pth",
#     tile_size=512,
#     tile_overlap=32,
#     use_fp16=True
# )